<center><h1>ipywebrtc</h1></center>

# Repository: https://github.com/maartenbreddels/ipywebrtc
# Installation:
`conda install -c conda-forge ipywebrtc`

# Usage:

Using `ipywebrtc` you can create a `MediaStream` out of:
- Any ipywidget using `WidgetStream`
- A video file using `VideoStream`
- An image file using `ImageStream`
- An audio file using `AudioStream`
- Your webcam/camera using `CameraStream`

From this `MediaStream` you can:
- Record a movie using `VideoRecorder`
- Record an image snapshot using `ImageRecorder`
- Record an audio fragment using `AudioRecorder`
- Stream it to peers using the simple `chat` function.

In [1]:
from ipywebrtc import CameraStream, ImageRecorder, VideoRecorder, AudioRecorder

## Display camera video stream

In [2]:
camera = CameraStream(constraints=
                      {'facing_mode': 'user',
                       'audio': False,
                       'video': { 'width': 640, 'height': 480 }
                       })
camera

CameraStream(constraints={'facing_mode': 'user', 'audio': False, 'video': {'width': 640, 'height': 480}})

## Capture image from the camera

In [3]:
ImageRecorder(stream=camera)

ImageRecorder(image=Image(value=b''), stream=CameraStream(constraints={'facing_mode': 'user', 'audio': False, …

## Capture audio

In [4]:
AudioRecorder(stream=camera)

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'facing_mode': 'user', '…

## Capture video

In [5]:
VideoRecorder(stream=camera)

VideoRecorder(stream=CameraStream(constraints={'facing_mode': 'user', 'audio': False, 'video': {'width': 640, …

## Chat

In [7]:
from ipywebrtc import chat, VideoStream, Video

In [8]:
room = chat(stream=VideoStream(video=Video.from_file('src/Big.Buck.Bunny.mp4')), room='QuantStack')

FileNotFoundError: [Errno 2] No such file or directory: 'src/Big.Buck.Bunny.mp4'

In [ ]:
# chat(room='QuantStack')

In [ ]:
image_recorder = ImageRecorder(stream=room.streams[1])
image_recorder

In [ ]:
from ipywidgets import Image, HBox
import PIL.Image
import io
import numpy as np
from skimage.filters import sobel
from skimage.color.adapt_rgb import adapt_rgb, each_channel
from skimage import filters

In [ ]:
out = Image()
stop_process = False

def process_image(_):
    if stop_process:
        return
    im_in = PIL.Image.open(io.BytesIO(image_recorder.image.value))
    im_array = np.array(im_in)[...,:3]
    im_array_edges = adapt_rgb(each_channel)(sobel)(im_array)
    im_array_edges = ((1-im_array_edges) * 255).astype(np.uint8)
    im_array_edges[im_array_edges < 250] = 0
    im_out = PIL.Image.fromarray(im_array_edges)
    f = io.BytesIO()
    im_out.save(f, format='png')
    out.value = f.getvalue()
    image_recorder.recording = True

image_recorder.image.observe(process_image, names=['value'])

HBox([out, image_recorder])

In [ ]:
stop_process = True

### Clean

In [ ]:
from ipywidgets import Widget
Widget.close_all()

## Capture stream from any widget

In [6]:
import ipyvolume
from sidecar import Sidecar
import numpy as np
from ipywebrtc import WidgetStream, VideoRecorder

ModuleNotFoundError: No module named 'ipyvolume'

In [ ]:
u = np.linspace(-10, 10, 25)
x, y = np.meshgrid(u, u)
r = np.sqrt(x**2+y**2)
x = x.flatten()
y = y.flatten()
r = r.flatten()
time = np.linspace(0, np.pi*2, 15)
z = np.array([(np.cos(r + t) * np.exp(-r/5)) for t in time])
color = np.array([[np.cos(r + t), 1-np.abs(z[i]), 0.1+z[i]*0] for i, t in enumerate(time)])
size = (z+1)
color = np.transpose(color, (0, 2, 1))

In [ ]:
sc = Sidecar(title='ipyvolume')

with sc:
    ipyvolume.figure()
    s = ipyvolume.scatter(x, z, y, color=color, marker="sphere")
    ipyvolume.animation_control(s, interval=200)
    ipyvolume.ylim(-3,3)
    ipyvolume.show()

In [ ]:
view_stream = WidgetStream(widget=ipyvolume.current.container)
view_stream

In [ ]:
recorder = VideoRecorder(stream=view_stream)
recorder

### Clean

In [ ]:
from ipywidgets import Widget
Widget.close_all()